#  Setup: Display options

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

#  Used to report timings
#

from timeit import default_timer

print("--")



--


# Setup: Connectivity

In [3]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [4]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [8]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 8v5HziwAJ9NPrBpMvME2MLfmSzN2u2MMn8KXvwqWq1PT, 1>


In [9]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


Database Name: my_db                          Id: 21JDRLcScJPzXu2CsX8hSCCwpuvBdXJbkLKQ7CdcKxV9
   Graph Name: my_graph                          Version: 1          Id: 8v5HziwAJ9NPrBpMvME2MLfmSzN2u2MMn8KXvwqWq1PT
   Graph Name: my_graph_BB                       Version: 10         Id: DfmNBHbvwBWi3PzzdrpaBEY7uQ69ofWFh3bwLURyDW1m
Database Name: default                        Id: 2ps7XNc4f8KLRVXwutJzqQVE3bTUdVbHw357hkA6PM5z

--


#  Step 00: Diagnostics

In [10]:

#  List all node types with node counts

l_query  = """

   MATCH (n) 
   WITH DISTINCT LABELS(n) AS labels, COUNT(n) AS cnt
   UNWIND labels AS label
   RETURN label, SUM(cnt) AS cnt
   ORDER BY label
   
   """.format()

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))


print()


#  List all edge types with counts

l_query  = """

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS type, COUNT(r) AS cnt
   RETURN type, cnt
   ORDER BY type
   
   """.format()

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))



          0/? [?op/s]

WorkerError: graph-worker exited while executing operation HteQrCuwxoirZS7AhoaMkEjzw6L8fPtGaRceKe1oJJnJ-4rBbgUwTs3xGPEA4Q (killed [origin_code: '3', extra_info: 'exit status 137: graph-worker exited with wait status 35072
']

In [ ]:

#  Return properties/keys from all vertices

l_query  = """
   MATCH (n)
   RETURN DISTINCT LABELS(n) AS label, KEYS(n) AS my_keys, COUNT(n) AS cnt
   """.format()

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))


#  Return properties/keys from all edges
#  

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT TYPE(r) AS my_type, KEYS(r) AS my_keys, COUNT(r) AS cnt
   """.format()

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))



#  Step 01: Actual Routines